<a href="https://colab.research.google.com/github/xin-2001/taica_1132_GenAI/blob/main/HW6/113_2GanAI_HW6_1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 主題一：使用HW5製作持續聊天的對話機器人
- 主題一
  延續上週作業，參考老師的範例，更改成可以持續對話的版本。
Gradio展示。
- 主題二
  
  製作兩個不同模型互相對話的機器人。
  
  Gradio展示。

- 繳交內容
  
  colab連結(請記得將共用權限打開)，請在重點處以 MarkDown 註明。
  
  此份作業的重點截圖
  
  人設/背景設定
  
  使用的模型
  
  Gradio的對話結果
  
  其他
  
  也可以自行增加其他內容

In [1]:
import os
from google.colab import userdata

Gemini模型可參考 https://ai.google.dev/gemini-api/docs/models?hl=zh-tw
- Gemini 2.5 Pro Experimental
- Gemini 2.0 Flash
- Gemini 2.0 Flash-Lite
- Gemini 1.5 Flash
- Gemini 1.5 Flash-8B
- Gemini 1.5 Pro

In [2]:
#【使用 Gemini】
api_key = userdata.get('GOOGLE_API_KEY')
model="gemini-1.5-flash" #
base_url="https://generativelanguage.googleapis.com/v1beta/openai/"

- os.environ: 這是一個字典，包含了目前環境的所有環境變數。
- ['OPENAI_API_KEY']: 這是在環境變數字典中，用來存取 OPENAI_API_KEY 這個環境變數的索引鍵。
- 透過 os.environ.get('OPENAI_API_KEY') 的方式來取得 API 金鑰，而不用直接在程式碼中寫入 API 金鑰。
- 優點
  - 提升安全性：避免將 API 金鑰直接寫入程式碼中，降低金鑰外洩的風險。
  - 提升程式碼可讀性：讓程式碼更簡潔，更容易理解。
  - 提升程式碼可維護性：方便修改 API 金鑰，而不用修改程式碼。

In [3]:
os.environ['OPENAI_API_KEY']=api_key

## 2. 程式的基本設定

1. 為機器人命名。

In [4]:
title = "柴語生成器"

2. 為對話機器人做角色設定。

 character 和 system 的差別
  1. system:
   - 用途：設定聊天機器人的整體行為和角色，例如它的個性、知識背景、說話風格等。
   - 側重點：定義聊天機器人的基本設定和限制，確保它在互動過程中保持一致性。
  2. character:
   - 用途：設定聊天機器人的特定角色或人格，例如它的名字、年齡、職業、興趣等。
   - 側重點：賦予聊天機器人更鮮明的個性，使其更具吸引力和趣味性。
  3. 總結
   - system 設定聊天機器人的整體行為和角色，而 character 設定聊天機器人的特定角色或人格。
   - system 更側重於定義基本設定和限制，而 character 更側重於賦予個性。
   - 可以根據你的需求和創意來組合使用 system 和 character，創造出獨特的聊天機器人。

In [5]:
system = '''你是一個友善的對話機器人，負責回答使用者的提問，回答需根據受試者的提問層次，從簡單回答開始，逐步深化內容，避免一次性提供完整答案。回答方式需以提示或建議為主，讓受試者透過提問獲取關鍵資訊。回答時請使用台灣用語及繁體中文。'''

3. 說明文字:讓使用者知道這是做什麼的對話機器人。

In [6]:
description = "我是友善的聊天機器人，可以回答妳各種問題。:)"

### 2. 使用 OpenAI 的 API

我們先來安裝 `openai` 套件, 還有快速打造 Web App 的 `gradio`。

In [7]:
!pip install openai
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.4 MB/s eta 0:00:00


### 3. 使用 ChatGPT API

1. 首先使用 `openai` 套件。

In [8]:
from openai import OpenAI

2. 貼上金鑰於colab的```Secret```
 - client: 這是一個變數，用來儲存 OpenAI 客戶端物件。
 - OpenAI(): 這是 openai 函式庫中的一個函式，用於初始化 OpenAI 客戶端。
 - base_url: 這是一個參數，用於指定 API 的基本網址。設定 base_url 可以讓你使用不同的 OpenAI API 服務，例如 OpenAI 官方服務或其他相容的服務。

In [9]:
client = OpenAI(
    base_url = base_url # 如用 OpenAI 不需要這一行
)

ChatGPT API 的重點是要把之前對話的內容送給 ChatGPT, 然後他就會有個適當的回應!

角色 (`role`) 一共有三種, 分別是:

* `system`: 這是對話機器人的「人設」
* `user`: 使用者
* `assistant`: ChatGPT 的回應

基本上過去的對話紀錄長這個樣子。

    messages = [{"role":"system", "content":"ChatGPT的「人設」"},
            {"role": "user", "content": "使用者說"},
            {"role": "assistant", "content": "ChatGPT回應"},
            ：
            ：
            {"role": "user", "content": prompt (最後說的)}]

## 4. 用 Gradio 打造你的對話機器人 Web App!

In [10]:
import gradio as gr

In [11]:
messages = [{"role":"system",
             "content":system}]

### 匯入多時區函式庫pytz

In [12]:
!pip install pytz==2023.3
import pytz #多時區資訊

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2


In [13]:
description2 = "我是友善的聊天機器人，可以回答妳各種問題。"
system2 = "你是一個友善的對話機器人，負責回答使用者的提問，回答需根據受試者的提問層次，從簡單回答開始，逐步深化內容，避免一次性提供完整答案。回答方式需以提示或建議為主，讓受試者透過提問獲取關鍵資訊。回答時請使用台灣用語及繁體中文。"

In [14]:
initial_messages = [{"role":"system",
             "content":system2},
            {"role":"assistant",
            'content':description2}]

In [15]:
state = gr.State(messages)

In [16]:
# 建立一個空的 list 來存放對話紀錄
chat_history = []

In [17]:
import datetime
def pipi(prompt, messages):
  user_id=userID
  taiwan_tz = pytz.timezone('Asia/Taipei')
  timestamp = datetime.datetime.now(taiwan_tz).strftime("%Y-%m-%d %H:%M:%S")  # 獲取時間戳記
  messages.append({"role": "user", "content": prompt, "timestamp": timestamp, "userID": user_id})
  chat_completion = client.chat.completions.create(
      messages=messages,
      model=model,
      )
  reply = chat_completion.choices[0].message.content
  messages.append({"role": "assistant", "content": reply, "timestamp": timestamp, "userID": user_id})
  chat_history.append([user_id, timestamp, "user", prompt])  # 使用者訊息
  chat_history.append([user_id, timestamp, "assistant", reply])  # 機器人訊息


  #history = history + [[prompt, reply]]
  return messages, messages

In [18]:
chatbot = gr.Chatbot(type="messages")

### 設置匯出按鈕及方式
- 檔名使用```16碼隨機ID_匯出的時間```

In [19]:
# 設置匯出按鈕及方式
from gradio import File
import csv
def export_chat_history(messages):
  user_id=userID
  # df = pd.DataFrame(messages)
  # df = df[['timestamp', 'role', 'content']]  # 選擇要匯出的欄位
  # csv_data = df.to_csv(index=False)  # 轉換為 CSV 格式
  taiwan_tz = pytz.timezone('Asia/Taipei')
  time_file = datetime.datetime.now(taiwan_tz).strftime("%Y-%m-%d %H:%M:%S")  # 獲取時間戳記
  filename = f"{user_id}_{time_file}.csv"

  with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['userID', 'timestamp', 'role', 'content'])  # 寫入標題列
    writer.writerows(chat_history)  # 寫入對話紀錄
  return filename
  # file_object = gr.File(label="CSV")
  # return file_object.download(value=csv_data, filename=f"chat_history_{time_file}.csv")
  #return gr.File(value=csv_data, filename=f"chat_history_{time_file}.csv", lebel="CSV")  # 返回 CSV 檔案

### 產生隨機ID

In [20]:
import random

def generate_random_hex(length=16):
  """產生指定長度的隨機 16 進位字串。
  Args:
    length: 16 進位字串的長度。預設為 16。
  Returns:
    隨機產生的 16 進位字串。
  """
  hex_characters = '0123456789abcdef'
  random_hex = ''.join(random.choice(hex_characters) for _ in range(length))
  return random_hex

# 產生 16 位數的 16 進位隨機亂碼
random_hex_code = generate_random_hex()
print(random_hex_code)  # 輸出範例: 'a1b2c3d4e5f67890'

68a7fe11ee930e23


In [ ]:
chat_history = [] #將對話紀錄清空
with gr.Blocks(title=title) as demo:
  gr.Markdown(f"## 🤖 {title}\n{description}")

  userID = generate_random_hex() # 產生使用者編號
  chatbot = gr.Chatbot(type="messages", value=initial_messages)

  msg = gr.Textbox(label="輸入訊息")
  #state_id = gr.State(test())
  #state = gr.State({"initial_messages": initial_messages.copy(), "user_id": None})
  state = gr.State(initial_messages.copy())  # 務必用 copy()
  # 清除送出後輸入框內容.then(fn=lambda x: "", outputs=[msg])
  msg.submit(fn=pipi, inputs=[msg, state], outputs=[chatbot, state]).then(fn=lambda x: "", outputs=[msg])
  # 設置匯出按鈕
  export_button = gr.Button("匯出聊天紀錄")
  file_output = gr.File(label="CSV")
  export_button.click(fn=export_chat_history, inputs=[state], outputs=[file_output])

demo.launch(share=True, debug=True)

/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1024: UserWarning: Expected 1 arguments for function <function <lambda> at 0x7ad1ae7431a0>, received 0.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1028: UserWarning: Expected at least 1 arguments for function <function <lambda> at 0x7ad1ae7431a0>, received 0.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://59bbfebe79fc195908.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/gradio/helpers.py:977: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")


## 成果展現
- 對話紀錄:[6a177f2dd0b1266a_2025-04-13%20190250.csv](https://github.com/xin-2001/taica_1132_GenAI/blob/a148781b9c6463f0314d4408d5dfad3691f2ce41/HW6/6a177f2dd0b1266a_2025-04-13%20190250.csv)
- 成果畫面
  > ![image.png](https://github.com/xin-2001/taica_1132_GenAI/blob/main/HW6/hw6_1_5_1.png?raw=true)